This is the required imports. We use the requests to request data form the API and we import the pandas and json_normalize to get better vision of the data, which makes it more easy to analyze from

In [9]:
import requests
import pandas as pd
from pandas import json_normalize

In [ ]:
import requests
import pandas as pd

API_TOKEN = "KuQ4Rt1ypOCvXfcm3cZXdPhOUlbuOrpHBgJkFm1MWTvtRR8TLhgEI02hjDxz"

ROUND_ID = 373227  # <-- put your round id here (example from your screenshot)

base_url = "https://api.sportmonks.com/v3/football"
endpoint = f"/rounds/{ROUND_ID}"

# Matches the screenshot
include = "fixtures.odds.market;fixtures.odds.bookmaker;fixtures.participants;league.country"
filters = "markets:1;bookmakers:2"

response = requests.get(
    f"{base_url}{endpoint}",
    params={
        "api_token": API_TOKEN,
        "include": include,
        "filters": filters,
    },
    timeout=30,
)

response.raise_for_status()
data = response.json()

# --- parse: round -> fixtures[] -> odds[] ---
round_data = data.get("data", {}) or {}
fixtures = round_data.get("fixtures", []) or []

rows = []
for fx in fixtures:
    fixture_name = fx.get("name") or str(fx.get("id"))
    starting_at = fx.get("starting_at")

    odds = fx.get("odds", []) or []
    for odd in odds:
        market = odd.get("market") or {}
        bookmaker = odd.get("bookmaker") or {}

        rows.append(
            {
                "Fixture": fixture_name,
                "StartingAt": starting_at,
                "OddID": odd.get("id"),
                "MarketID": odd.get("market_id"),
                "Market": market.get("name"),
                "BookmakerID": odd.get("bookmaker_id"),
                "Bookmaker": bookmaker.get("name"),
                "Label": odd.get("label"),   # e.g., Home/Draw/Away (depends on market)
                "Odds": odd.get("value"),    # decimal odds as string/number
            }
        )

df = pd.DataFrame(rows)
df


,Fixture,StartingAt,OddID,MarketID,Market,BookmakerID,Bookmaker,Label,Odds
0,Mallorca vs Girona,2026-01-04 17:30:00,209275551981,1,Fulltime Result,2,bet365,Away,3.60
1,Mallorca vs Girona,2026-01-04 17:30:00,209275551979,1,Fulltime Result,2,bet365,Draw,3.10
2,Mallorca vs Girona,2026-01-04 17:30:00,209275551977,1,Fulltime Result,2,bet365,Home,2.20
3,Rayo Vallecano vs Getafe,2026-01-02 20:00:00,209270514400,1,Fulltime Result,2,bet365,Away,4.33
4,Rayo Vallecano vs Getafe,2026-01-02 20:00:00,209270514399,1,Fulltime Result,2,bet365,Draw,2.87
5,Rayo Vallecano vs Getafe,2026-01-02 20:00:00,209270514398,1,Fulltime Result,2,bet365,Home,2.10
6,Elche vs Villarreal,2026-01-03 17:30:00,209270545293,1,Fulltime Result,2,bet365,Away,2.25
7,Elche vs Villarreal,2026-01-03 17:30:00,209270545290,1,Fulltime Result,2,bet365,Draw,3.40
8,Elche vs Villarreal,2026-01-03 17:30:00,209270545282,1,Fulltime Result,2,bet365,Home,3.10
9,Celta de Vigo vs Valencia,2026-01-03 13:00:00,209270545050,1,Fulltime Result,2,bet365,Away,3.90


In [ ]:
import requests
import pandas as pd
from datetime import date, timedelta

API_TOKEN = "KuQ4Rt1ypOCvXfcm3cZXdPhOUlbuOrpHBgJkFm1MWTvtRR8TLhgEI02hjDxz"
BASE_URL = "https://api.sportmonks.com/v3/football"

# --- next 3 full days (starting tomorrow) ---
start_date = date.today() + timedelta(days=1)
end_date = start_date + timedelta(days=2)  # 3 days total: start_date, +1, +2

endpoint = f"/fixtures/between/{start_date:%Y-%m-%d}/{end_date:%Y-%m-%d}"

# Includes similar to your screenshot, but for fixtures directly (not rounds)
# NOTE: We intentionally do NOT pass `filters` so you get ALL markets/bookmakers available.
include = "odds.market;odds.bookmaker;participants;league.country"

session = requests.Session()

all_fixtures = []
page = 1

while True:
    resp = session.get(
        f"{BASE_URL}{endpoint}",
        params={
            "api_token": API_TOKEN,
            "include": include,
            "page": page,  # pagination
        },
        timeout=30,
    )
    resp.raise_for_status()
    payload = resp.json()

    fixtures_page = (payload.get("data") or [])
    all_fixtures.extend(fixtures_page)

    # SportMonks v3 responses include pagination metadata; stop when there is no next page.
    pagination = (payload.get("meta") or {}).get("pagination") or {}
    if not pagination or pagination.get("current_page") >= pagination.get("total_pages", 0):
        break

    page += 1

# --- parse: fixtures[] -> odds[] ---
rows = []
for fx in all_fixtures:
    fixture_name = fx.get("name") or str(fx.get("id"))
    starting_at = fx.get("starting_at")

    odds = fx.get("odds") or []
    for odd in odds:
        market = odd.get("market") or {}
        bookmaker = odd.get("bookmaker") or {}

        rows.append(
            {
                "Fixture": fixture_name,
                "StartingAt": starting_at,
                "OddID": odd.get("id"),
                "MarketID": odd.get("market_id"),
                "Market": market.get("name"),
                "BookmakerID": odd.get("bookmaker_id"),
                "Bookmaker": bookmaker.get("name"),
                "Label": odd.get("label"),
                "Odds": odd.get("value"),
            }
        )

df = pd.DataFrame(rows)
df

,Fixture,StartingAt,OddID,MarketID,Market,BookmakerID,Bookmaker,Label,Odds
0,Bologna vs Atalanta,2026-01-07 17:30:00,211220109017,268,Player Total Shots,2,bet365,1.5,1.28
1,Bologna vs Atalanta,2026-01-07 17:30:00,210906230540,57,Correct Score,2,bet365,1,126.00
2,Bologna vs Atalanta,2026-01-07 17:30:00,210921900233,90,Goalscorers,2,bet365,First,9.50
3,Bologna vs Atalanta,2026-01-07 17:30:00,210906230419,104,Alternative Asian Handicap,2,bet365,2,3.10
4,Bologna vs Atalanta,2026-01-07 17:30:00,210910604537,86,Team Total Goals,2,bet365,2,1.53
...,...,...,...,...,...,...,...,...,...
81658,Getafe vs Real Sociedad,2026-01-09 20:00:00,210260989611,94,Alternative Handicap Result,2,bet365,2,17.00
81659,Getafe vs Real Sociedad,2026-01-09 20:00:00,210253734722,127,Specials,2,bet365,2,3.50
81660,Getafe vs Real Sociedad,2026-01-09 20:00:00,210259430485,332,Player to Assist,2,bet365,Assist,7.00
81661,Getafe vs Real Sociedad,2026-01-09 20:00:00,210261716744,92,Team Goalscorer,2,bet365,Last,5.50


In [20]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

API_TOKEN = "KuQ4Rt1ypOCvXfcm3cZXdPhOUlbuOrpHBgJkFm1MWTvtRR8TLhgEI02hjDxz"
BASE_URL = "https://api.sportmonks.com/v3/football"

INCLUDE = "odds.market;odds.bookmaker;participants;league.country"


def fetch_odds_next_12h() -> pd.DataFrame:
    now_utc = datetime.now(timezone.utc)
    end_utc = now_utc + timedelta(hours=12)

    # SportMonks is date-based → fetch today + tomorrow
    start_date = now_utc.date()
    end_date = (now_utc + timedelta(days=1)).date()
    endpoint = f"/fixtures/between/{start_date:%Y-%m-%d}/{end_date:%Y-%m-%d}"

    session = requests.Session()
    all_fixtures = []
    page = 1

    while True:
        resp = session.get(
            f"{BASE_URL}{endpoint}",
            params={"api_token": API_TOKEN, "include": INCLUDE, "page": page},
            timeout=30,
        )
        resp.raise_for_status()
        payload = resp.json()

        all_fixtures.extend(payload.get("data") or [])

        pagination = ((payload.get("meta") or {}).get("pagination")) or {}
        cur = pagination.get("current_page")
        tot = pagination.get("total_pages")
        if cur is None or tot is None or int(cur) >= int(tot):
            break
        page += 1

    rows = []
    for fx in all_fixtures:
        odds = fx.get("odds") or []
        for odd in odds:
            rows.append(
                {
                    "FixtureID": fx.get("id"),
                    "Fixture": fx.get("name"),
                    "StartingAt": fx.get("starting_at"),
                    "MarketID": odd.get("market_id"),
                    "Market": (odd.get("market") or {}).get("name"),
                    "BookmakerID": odd.get("bookmaker_id"),
                    "Bookmaker": (odd.get("bookmaker") or {}).get("name"),
                    "Label": odd.get("label"),
                    "Odds": odd.get("value"),
                }
            )

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    df["StartingAt"] = pd.to_datetime(df["StartingAt"], utc=True, errors="coerce")
    df["Odds"] = pd.to_numeric(df["Odds"], errors="coerce")

    # Filter strictly to now → +12h
    df = df[df["StartingAt"].between(now_utc, end_utc)]

    return df.dropna(subset=["Odds", "MarketID", "BookmakerID", "Label"])

In [21]:
import numpy as np


def find_valuebets(
    df: pd.DataFrame,
    min_books: int = 4,
    z_thresh: float = 2.5,
    min_edge_pct: float = 3.0,
) -> pd.DataFrame:
    if df.empty:
        return df.copy()

    work = df[df["Odds"] > 1.0].copy()
    group_cols = ["FixtureID", "MarketID", "Label"]

    def detect(g):
        if g["BookmakerID"].nunique() < min_books:
            return None

        x = np.log(g["Odds"].to_numpy())
        med = np.median(x)
        mad = np.median(np.abs(x - med))
        if mad == 0:
            return None

        z = (x - med) / (1.4826 * mad)
        g = g.copy()
        g["ConsensusOdds"] = float(np.exp(med))
        g["EdgePct_vsMedian"] = (g["Odds"] / g["ConsensusOdds"] - 1) * 100
        g["RobustZ"] = z

        return g[(g["RobustZ"] >= z_thresh) & (g["EdgePct_vsMedian"] >= min_edge_pct)]

    flagged = (
        work.groupby(group_cols, group_keys=False)
        .apply(detect)
        .dropna(how="all")
        .reset_index(drop=True)
    )

    return flagged.sort_values("EdgePct_vsMedian", ascending=False)

In [22]:
df = fetch_odds_next_12h()
valuebets = find_valuebets(df)

valuebets[
    [
        "Fixture",
        "StartingAt",
        "Market",
        "Label",
        "Bookmaker",
        "Odds",
        "ConsensusOdds",
        "EdgePct_vsMedian",
        "RobustZ",
    ]
]

,Fixture,StartingAt,Market,Label,Bookmaker,Odds,ConsensusOdds,EdgePct_vsMedian,RobustZ
111,Sassuolo vs Juventus,2026-01-06 19:45:00+00:00,Handicap Result,Home,Pinnacle,115.77,2.154994,5272.172233,4.133323
166,West Ham United vs Nottingham Forest,2026-01-06 20:00:00+00:00,Handicap Result,Home,Pinnacle,65.51,1.644992,3882.389217,5.417787
14,Lecce vs Roma,2026-01-06 17:00:00+00:00,Handicap Result,Home,Pinnacle,57.18,2.040000,2702.941176,3.749450
258,Rangers vs Aberdeen,2026-01-06 20:00:00+00:00,1st Half Goals,Over,1xbet,65.00,2.374995,2636.848170,4.052193
54,Pisa vs Como,2026-01-06 14:00:00+00:00,Handicap Result,Home,Pinnacle,55.12,2.020000,2628.712871,3.781391
...,...,...,...,...,...,...,...,...,...
79,Sassuolo vs Juventus,2026-01-06 19:45:00+00:00,Fulltime Result,Away,1xbet,1.76,1.700000,3.529412,2.638418
0,Lecce vs Roma,2026-01-06 17:00:00+00:00,Fulltime Result,Away,1xbet,1.80,1.739971,3.449984,2.661341
168,West Ham United vs Nottingham Forest,2026-01-06 20:00:00+00:00,Correct Score,1:0,Marathonbet,9.30,9.000000,3.333333,3.328511
49,Pisa vs Como,2026-01-06 14:00:00+00:00,Double Chance - 2nd Half,Draw/Away,WilliamHill,1.17,1.134989,3.084701,4.651556


## Unikke Bookmakers

In [17]:
import requests
import pandas as pd
from datetime import date, timedelta

API_TOKEN = "KuQ4Rt1ypOCvXfcm3cZXdPhOUlbuOrpHBgJkFm1MWTvtRR8TLhgEI02hjDxz"
BASE_URL = "https://api.sportmonks.com/v3/football"

start_date = date.today() + timedelta(days=1)
end_date = start_date + timedelta(days=2)

endpoint = f"/fixtures/between/{start_date:%Y-%m-%d}/{end_date:%Y-%m-%d}"
include = "odds.bookmaker"

session = requests.Session()
bookmakers = {}
page = 1

while True:
    response = session.get(
        f"{BASE_URL}{endpoint}",
        params={
            "api_token": API_TOKEN,
            "include": include,
            "page": page,
        },
        timeout=30,
    )
    response.raise_for_status()
    data = response.json()

    fixtures = data.get("data") or []

    for fx in fixtures:
        for odd in (fx.get("odds") or []):
            bm = odd.get("bookmaker")
            if bm and bm.get("id") is not None:
                bookmakers[bm.get("id")] = {
                    "BookmakerID": bm.get("id"),
                    "Name": bm.get("name"),
                    "Slug": bm.get("slug"),
                }

    # --- robust pagination ---
    pagination = ((data.get("meta") or {}).get("pagination")) or {}
    current_page = pagination.get("current_page")
    total_pages = pagination.get("total_pages")

    # If the API doesn't return pagination info, assume single page.
    if current_page is None or total_pages is None:
        break

    if int(current_page) >= int(total_pages):
        break

    page += 1

df_bookmakers = (
    pd.DataFrame(bookmakers.values())
    .drop_duplicates(subset=["BookmakerID"])
    .sort_values("Name", na_position="last")
    .reset_index(drop=True)
)

df_bookmakers

,BookmakerID,Name,Slug
0,1,10Bet,None
1,3,188Bet,None
2,35,1xbet,None
3,5,888Sport,None
4,9,Betfair,None
5,26,Betsson,None
6,38,CloudBet,None
7,14,Dafabet,None
8,59,HKJC,None
9,27,Interwetten,None
